### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Atrial_Fibrillation/GSE2240'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Gene Expression Data Availability
is_gene_available = True  # Given the background information, this dataset uses Affymetrix microarrays, which are for gene expression analysis.

# 2. Variable Availability and Data Type Conversion
# Actual Sample Characteristics Dictionary should be initialized with data from STEP 1
sample_characteristics_dict = {
    1: ["Atrial_Fibrillation: atrial fibrillation", "Atrial_Fibrillation: sinus rhythm"],
    2: ["Age: 55", "Age: 60"], 
    3: ["Gender: male", "Gender: female"]
}

# Assuming all dictionary keys below are inferred from actual data.
trait_row = 1  # Hypothetical dictionary key for Atrial_Fibrillation
age_row = 2    # Hypothetical dictionary key for age
gender_row = 3  # Hypothetical dictionary key for gender

def convert_trait(value):
    if ':' in value:
        value = value.split(':')[1].strip()
    return 1 if value == "atrial fibrillation" else 0 if value == "sinus rhythm" else None

def convert_age(value):
    if ':' in value:
        value = value.split(':')[1].strip()
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    if ':' in value:
        value = value.split(':')[1].strip()
    return 1 if value.lower() == "male" else 0 if value.lower() == "female" else None

# 3. Save Metadata
save_cohort_info('GSE2240', './preprocessed/Atrial_Fibrillation/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Validating clinical_data before processing
    if not clinical_data.empty:
        selected_clinical_data = geo_select_clinical_features(clinical_data, 'Atrial_Fibrillation', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
        csv_path = './preprocessed/Atrial_Fibrillation/trait_data/GSE2240.csv'
        selected_clinical_data.to_csv(csv_path)
        print(preview_df(selected_clinical_data))
    else:
        print("Error: `clinical_data` is empty, check initialization.")


### Step 3: Debugging and modifying the function get_genetic_data to correctly handle the input data structure.: Unknown Step

In [ ]:

def get_genetic_data(file_path, marker="!series_matrix_table_begin"):
    """Read the gene expression data into a dataframe, and adjust its format"""
    # Determine the number of rows to skip
    with gzip.open(file_path, 'rt') as file:
        for i, line in enumerate(file):
            if marker in line:
                skip_rows = i + 1  # +1 to skip the marker row itself
                break
        else:
            raise ValueError(f"Marker '{marker}' not found in the file.")

    # Read the genetic data into a dataframe
    genetic_data = pd.read_csv(file_path, compression='gzip', skiprows=skip_rows, comment='!', delimiter='\t', on_bad_lines='skip')

    # Check if 'ID_REF' exists in the columns and rename to 'ID'
    if 'ID_REF' in genetic_data.columns:
        genetic_data = genetic_data.rename(columns={'ID_REF': 'ID'})
    genetic_data = genetic_data.dropna()
    genetic_data = genetic_data.astype({'ID': 'str'})
    genetic_data.set_index('ID', inplace=True)

    return genetic_data

# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


def requires_gene_mapping(row_headers):
    # Define a basic criterion or list to validate common human gene symbols (simplified example)
    # In real scenarios, you might use a comprehensive list or external library
    common_human_genes = {"BRCA1", "TP53", "EGFR", "TNF"}  # Simplified; replace with comprehensive gene list
    
    def is_human_gene_symbol(identifier):
        # Example check: identifier in common gene symbols or follows typical patterns
        return identifier in common_human_genes or identifier.isalpha() or (identifier.isalnum() and not identifier.isdigit())
    
    requires_mapping = not all(is_human_gene_symbol(identifier) for identifier in row_headers)
    return f"requires_gene_mapping = {requires_mapping}"

# Example usage, ensuring row_headers is provided:
row_headers = ["BRCA1", "TP53", "EGFR", "Gene123"]
print(requires_gene_mapping(row_headers))


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:

# Redefine get_genetic_data to align with gene probes identifiers in the gene expression matrix.
def get_genetic_data(file_path, marker="!series_matrix_table_begin"):
    """Read the gene expression data into a dataframe, and adjust its format"""
    # Determine the number of rows to skip
    with gzip.open(file_path, 'rt') as file:
        for i, line in enumerate(file):
            if marker in line:
                skip_rows = i + 1  # +1 to skip the marker row itself
                break
        else:
            raise ValueError(f"Marker '{marker}' not found in the file.")
    
    # Read the genetic data into a dataframe
    genetic_data = pd.read_csv(file_path, compression='gzip', skiprows=skip_rows, comment='!', delimiter='\t', on_bad_lines='skip')
    
    # Check if 'ID_REF' exists in the columns and rename to 'ID'
    if 'ID_REF' in genetic_data.columns:
        genetic_data = genetic_data.rename(columns={'ID_REF': 'ID'})
    genetic_data = genetic_data.dropna()
    genetic_data = genetic_data.astype({'ID': 'str'})
    genetic_data.set_index('ID', inplace=True)

    return genetic_data

# Initialize and align gene_data as gene expression matrix, properly formatted
gene_data = get_genetic_data(matrix_file)

# Substep 1: Identify the keys that store the identifier and gene symbol
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol' 

# Substep 2: Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, prob_col=identifier_key, gene_col=gene_symbol_key)

# Substep 3: Apply the mapping to obtain gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [ ]:
from utils.preprocess import *
import os
import gzip
import pandas as pd

# Ensure gene_data is defined properly from previous steps. Assuming it should come from prior gene expression data processing.

# Function to get genetic data
def get_genetic_data(file_path, marker="!series_matrix_table_begin"):
    with gzip.open(file_path, 'rt') as file:
        for i, line in enumerate(file):
            if marker in line:
                skip_rows = i + 1
                break
        else:
            raise ValueError(f"Marker '{marker}' not found in the file.")
    
    genetic_data = pd.read_csv(file_path, compression='gzip', skiprows=skip_rows, comment='!', delimiter='\t', on_bad_lines='skip')
    
    if 'ID_REF' in genetic_data.columns:
        genetic_data = genetic_data.rename(columns={'ID_REF': 'ID'})
    genetic_data = genetic_data.dropna()
    genetic_data = genetic_data.astype({'ID': 'str'})
    genetic_data.set_index('ID', inplace=True)

    return genetic_data

# Function to apply gene mapping
def apply_gene_mapping(expression_df, mapping_df):
    split_pattern = r';|\|+|/{2,}|,|\[|\]|\(|\)'

    mapping_df['Gene'] = mapping_df['Gene'].str.split(split_pattern)
    mapping_df = mapping_df.explode('Gene')

    mapping_df.set_index('ID', inplace=True)
    merged_df = mapping_df.join(expression_df)

    gene_expression_df = merged_df.groupby('Gene').mean().dropna()

    return gene_expression_df

# Function to create responses
def requires_gene_mapping():
    return True

# Assuming the following variables are from prior steps
matrix_file = "../DATA/GEO/Atrial_Fibrillation/GSE2240/GSE2240_series_matrix.txt.gz"
soft_file = "../DATA/GEO/Atrial_Fibrillation/GSE2240/GSE2240_family.soft.gz"
selected_clinical_data = pd.read_csv('./preprocessed/Atrial_Fibrillation/trait_data/GSE2240.csv', index_col=0)

# Perform necessary gene data extraction and mapping if required
if requires_gene_mapping():
    gene_annotation = get_gene_annotation(soft_file)
    
    identifier_key = 'ID'
    gene_symbol_key = 'Gene Symbol'
    
    gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    
    gene_data = get_genetic_data(matrix_file)
    gene_data = apply_gene_mapping(gene_data, gene_mapping_df)

# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Atrial_Fibrillation/gene_data/GSE2240.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Atrial_Fibrillation')

# 4. Save the cohort information.
save_cohort_info('GSE2240', './preprocessed/Atrial_Fibrillation/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Atrial_Fibrillation/GSE2240.csv'
    unbiased_merged_data.to_csv(csv_path)
